In [31]:
%matplotlib inline

In [ ]:
import pandas as pds
from pathlib import Path
import numpy as np

In [ ]:
# This notebook makes MODIS and GMB timeseries files on the desired format for LamaH-Ice dataset publication

# First we process the MODIS timeseries

In [33]:
# Read MODIS timeseries
path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\2_timeseries\daily")
full_watershed = path/'modis_fsca_full_watershed_final.csv'
outside_glaciers = path/'modis_fsca_outside_glaciers.csv'
glacier_albedo = path/'modis_glacier_albedo.csv'

df_full = pds.read_csv(full_watershed)
df_full = df_full.set_index('date_time')

df_outside_g = pds.read_csv(outside_glaciers)
df_outside_g = df_outside_g.set_index('date_time')

df_gl_alb = pds.read_csv(glacier_albedo)
df_gl_alb = df_gl_alb.set_index('date_time')

save_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\2_timeseries\daily\modis_fractional_snow_cover_and_glacier_albedo")

In [40]:
for i in df_full.columns:
    if i in [ '20000', '20001', '20002']:
        continue
    full = df_full[i]
    outside = df_outside_g[i]
    try:
        glacier = df_gl_alb[i]
    except KeyError:
        glacier = outside*np.nan
    
    combined = pds.concat([full,outside,glacier],axis=1)
    combined.columns = ['fsca_full_watershed','fsca_outside_glaciers','glacier_albedo']
    save_name = '%s.csv' % i
    combined.to_csv(save_path/save_name)
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
890
990
1010
9900


# Now we process the GMB timeseries in a similar manner

In [48]:
# Read glacier annual timeseries
path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\2_timeseries\annual")
annual_net_MB = path/'GMB_annual_net.csv'
summer_MB = path/'GMB_summer.csv'
winter_MB = path/'GMB_winter.csv'
glac_area_perc = path/'glaciated_area_percentage.csv'
glac_area_km2 = path/'glaciated_area_km2.csv'
save_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\2_timeseries\annual")

In [87]:
import pandas as pd
import numpy as np

# Read all input CSV files
df_annual_net = pd.read_csv(annual_net_MB, index_col='Unnamed: 0')
df_summer = pd.read_csv(summer_MB, index_col='Unnamed: 0')
df_winter = pd.read_csv(winter_MB, index_col='Unnamed: 0')
df_glac_area_perc = pd.read_csv(glac_area_perc, index_col='id')
df_glac_area_perc = df_glac_area_perc.transpose()
df_glac_area_km2 = pd.read_csv(glac_area_km2, index_col='id')
df_glac_area_km2 = df_glac_area_km2.transpose()

# Convert mass balance dataframe indices to strings
df_annual_net.index = df_annual_net.index.astype(str)
df_summer.index = df_summer.index.astype(str)
df_winter.index = df_winter.index.astype(str)

# Get a list of unique catchment IDs
catchment_ids = df_glac_area_km2.columns

# Loop through catchment IDs and create separate CSV files
for catchment_id in catchment_ids:
    try:
        combined_data = pd.DataFrame({
            'annual_net_MB': df_annual_net[str(catchment_id)],
            'summer_MB': df_summer[str(catchment_id)],
            'winter_MB': df_winter[str(catchment_id)],
            'glac_area_perc': df_glac_area_perc[catchment_id],
            'glac_area_km2': df_glac_area_km2[catchment_id]
        })
        # Fill missing values for years 2020, 2021, and 2022 with their 2019 value
        combined_data.loc['2020':'2022', ['glac_area_perc', 'glac_area_km2']] = combined_data.loc['2019', ['glac_area_perc', 'glac_area_km2']].values
    except KeyError:
        combined_data = pd.DataFrame({
            'annual_net_MB': df_glac_area_perc[catchment_id]*np.nan,
            'summer_MB': df_glac_area_perc[catchment_id]*np.nan,
            'winter_MB': df_glac_area_perc[catchment_id]*np.nan,
            'glac_area_perc': df_glac_area_perc[catchment_id],
            'glac_area_km2': df_glac_area_km2[catchment_id]
        })
        # Fill missing values for years 2020, 2021, and 2022 with their 2019 value
        combined_data.loc['2020':'2022', ['glac_area_perc', 'glac_area_km2']] = combined_data.loc['2019', ['glac_area_perc', 'glac_area_km2']].values
    
    # Save the combined data to a CSV file
    output_filename = save_path / f'{catchment_id}.csv'
    combined_data.to_csv(output_filename,sep=';')

print("CSV files created successfully.")


CSV files created successfully.
